<a href="https://colab.research.google.com/github/alassou/TD-Up2/blob/master/Copie_de_Copie_de_TD_5456.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[texte du lien](https://)# **Twitter TD 6_G**

Dates: 2019



## Initial Setup

- **Run "Setup" below first.**

    - This will load libraries and download some resources that we'll use throughout the tutorial.

    - You will see a message reading "Done with setup!" when this process completes.


In [ ]:
#@title Setup (click the "run" button to the left) {display-mode: "form"}

## Setup ##

# imports

# built-in Python libraries
# -------------------------

! pip install peony-twitter[all]
! pip install peony-twitter[all]
! pip show tornado
! pip install nest_asyncio 
import nest_asyncio
nest_asyncio.apply()
! pip install yahoo_fin
! pip install requests_html
import requests
from bs4 import BeautifulSoup
import requests_html
import lxml.html as lh
import pandas as pd
import re
from datetime import datetime
from datetime import timedelta
import time
import json
import datetime
from peony import EventStream, PeonyClient, event_handler, events
import asyncio
import random
import sys
import csv
import requests

print("Done with setup!")
print("If you'd like, you can click the (X) button to the left to clear this output.")

# generate graph from twitter json
def create_graph(data):
    all_data = dict(data)
    edge_list = []
    user_screen_name = all_data['user']['screen_name']
    timestamp_ms = all_data['timestamp_ms']
    if len(all_data['entities']['hashtags']) > 0:
        for index, i in enumerate(all_data['entities']['hashtags']):
            edge_list.append(("@"+user_screen_name, "#"+all_data['entities']['hashtags'][index]['text'], timestamp_ms))
    if len(all_data['entities']['user_mentions']) > 0:
        for index, i in enumerate(all_data['entities']['user_mentions']):
            edge_list.append(("@"+user_screen_name, "@"+all_data['entities']['user_mentions'][index]['screen_name'], timestamp_ms))
    return edge_list
  
# compute components from reservoir of edges
def components(edges):
  global d
  d={}
  n=0
  for e in edges:
     a =e [0]
     b = e[1]
     if a in d.keys():
        d[a].add(b)
     else:
      d[a]=set([b])
     if b in d.keys():
      d[b].add(a)
     else:
      d[b]=set([a])

  #print ("Dict=", d, len(d))
  n=len(d)
  m=0
  #Breadth-first search from first point, first component
  a=list(d.keys())[0]

  #dc keeps b:i  if i is the length of the shortest path from a to b in the first component
  dc={}
  dc[a]=0
  for b in d[a]: 
    if b in dc.keys(): pass
    else: dc[b]=1

  #Initialize S ans S1, Start iterating
  S=d[a]
  comp=[]
  S1=set([a])
  S=S.union(S1)
  #print("S=",S)
  while S > S1:
      S1=S
      for u in S:
         S=S.union(d[u])
         for v in d[u]:
           if v in dc.keys():
              dc[v]=min(dc[v],dc[u]+1)
           else:
            dc[v]=dc[u]+1
  for u in S:
       m=m+len(d[u])

  comp.append((len(S),int(m/2),list(S)))
    
  #print("Component",comp)

  ST=S
    
  #print("ST=",ST)

  #The other components: origin must be outside ST, same treatment
  i=1
  while i<len(d):
   m=0
   while  list(d.keys())[i] not in ST:
    a=list(d.keys())[i]
    S=d[a]
    S1=set([a])
    S=S.union(S1)
    while S > S1:
      S1=S
      for u in S:
         S=S.union(d[u])
    for u in S:
       m=m+len(d[u])
    comp.append((len(S),int(m/2),list(S)))
    ST=ST.union(S)  
   i+=1     
  return comp


def comp_edges(comp1):
 compedges=[]
 j0=0
 while j0<len(comp1):
   l1 = comp1[j0][2]
   j=0
   cp=[]
   while j <len(l1):
     a=l1[j]
     j1=0
     while j1< len(d[l1[j]]):
      f=(a,list(d[l1[j]])[j1])
      cp.append(f)
      j1 +=1
     j +=1
   j0 +=1
   compedges.append(cp)
 return compedges

# write edges into file

def write_edge_reservoir(i,time_export):
  with open('sample_data/%s_step_reservoir_edges.csv' % (time_export), 'w') as f:
    f.write("window_counter,vertices,edges,Diameter,bitcoin_value\n")
    writer = csv.writer(f, delimiter=',')
    writer.writerows(i)
    if time.time()*1000 > start + timeout:
     f.close()
    else:
     print("")
  return True

def write_edge_reservoir1(i, time_export):
    f = open("sample_data/%s_step_reservoir_edges.csv" % time_export, "a")
    f.write("Diameter,vertices,edges,time,bitcoin_value\n")
    # f.write("Source, Destination \n")
    for j in i:
      writer = csv.writer(f, delimiter=',')
      writer.writerows(i)
      f.close()
    return True

Name: tornado
Version: 5.1.1
Summary: Tornado is a Python web framework and asynchronous networking library, originally developed at FriendFeed.
Home-page: http://www.tornadoweb.org/
Author: Facebook
Author-email: python-tornado@googlegroups.com
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: terminado, notebook, jupyter-client, ipykernel, google-colab, distributed, bokeh
     |████████████████████████████████| 143kB 5.6MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Done with setup!
If you'd like, you can click the (X) button to the left to clear this output.


## The Twitter API 

- Twitter is also known for being an abundant source of publc text data (perhaps even more so than Reddit).
- Twitter provides several types of API that can be used to collect anything from tweets to user descriptions to follower networks.
    - You can [read all about it here](https://developer.twitter.com/).
- For this tutorial, we'll look at using the [streaming API](https://developer.twitter.com/en/docs/tutorials/consuming-streaming-data.html), which allows us to retreive tweets that contain specific words, phrases, and hashtags.
- In the slides, we talked about how to setup a Twitter App and get a API keys.
    - You should add your own keys below and then run the code block to set your keys:
    
#### [Tutorial link to create an Twitter app](https://botwiki.org/resource/tutorial/how-to-create-a-twitter-app/) or https://www.youtube.com/watch?v=aB1x7ZsbbaQ&feature=youtu.be

In [ ]:

# Go to http://dev.twitter.com and create an app. 
# The consumer key and secret will be generated for you after
consumer_key="MfKJZNJdN2cVM6Ka8xmF8Smbk"
consumer_secret="KtpAZ0sK0HdpVphCryNWTlmWwnm97X3rovkrCwOOOtGbVL2Zfo"

# After the step above, you will be redirected to your app's page.
# Create an access token under the the "Your access token" section
access_token="2207034565-3LYXBvDdzeDMXpmM4i3APZkG6qW6UGeTc8xQKHe"
access_token_secret="3jyb0b9f5gGA8sTJbq2j5IWwDAEBC7G0EBMYvb8WIpONQ"


- Do not share your credentials with anyone!
    - You shouldn't hardcode your API keys in code (like above) if you are going to save the file anywhere that is visible to others (like commiting the file to github).
        - You can read more about securing your API keys [here](https://developer.twitter.com/en/docs/basics/authentication/guides/securing-keys-and-tokens).
     - So, if you plan to save this file in any way, make sure to remove your API keys first.
     - If you think your keys have been compromized, you can regenerate them.
        - [Apps](https://developer.twitter.com/en/apps) -> Keys and Tokens -> Regenerate

## Capture Data

![Step reservoir sampling](https://github.com/GuillaumeVIMONT/TD_UP2_new/blob/master/static/img/continuous_sampling.png?raw=true)

Before start Twitter capture we need some informations :

* $k$ is the size of the reservoir \\
* $tau$ is the lenght of the window \\
* $lambda$ is the lenght of a strate \\
* $tracking$ is the keywords to filter Twitter stream \\
* $threshold$ is the threshold for the connected component export \\
* $timeout$ is the duration of the capture

In [ ]:
#@tickers (double-click to view) {display-mode: "form"}
ticker = "BTC-EUR" #@param {type:"string"}
ticker1 = "^IXIC" #@param {type:"string"}


In [ ]:
#@title Capture Twitter streaming (double-click to view) {display-mode: "form"}

k = 400 #@param {type:"number"}
tau = 3 #@param {type:"number"}
lamb = 1 #@param {type:"number"}
tracking = "cnn" #@param {type:"string"}
threshold = 9 #@param {type:"slider", min:1, max:20, step:1}
timeout = 68 #@param {type:"slider", min:1, max:120, step:1}
timeout = int(timeout)*60000
start = int(round(time.time() * 1000))

global L, sample_window_stream, global_counter, time_counter, interval_counter
L=0
sample_window_stream = []
global_counter = 0
interval_counter = 0
time_counter = (time.time()*1000)+60000


# k is the size of the reservoir (number of edges into the reservoir)


# tau is the lenght of the window
tau =  tau*60000

# lamb is the lenght of strate floato the window
lamb = lamb*60000

# rate is the number of strates per window
rate = tau/lamb

# M contain steps floato the window, for each strates we have a edges counter
M = [0] * int(rate)

# init it's a default parameters to configure somes variables when the reservoir strart
init = 0

reservoir = []
# Using graph_objects
import plotly.graph_objects as go

import pandas as pd
def diameter(i):
 comp = components(reservoir)
 comp1 = sorted(comp, reverse=True)
 #print("number of vertices in largest component",comp1[0][0])
 #print("number of edges in largest component",comp1[0][1])
 first=comp1[i]
 l=first[2]
 a=l[0]
#print("First point",a)
 dc={}
 dc[a]=0
 for b in d[a]: 
  if b in dc.keys(): pass
  else: dc[b]=1
  
#Initialize S ans S1, Start iterating
 S=d[a]
 comp=[]
 S1=set([a])
 S=S.union(S1)
#print("S=",S)
 while S > S1:
    S1=S
    for u in S:
       S=S.union(d[u])
       for v in d[u]:
         if v in dc.keys(): dc[v]=min(dc[v],dc[u]+1)
         else: dc[v]=dc[u]+1
 dist=list(dc.values())
 ma=max(dist)
 for a, i in dc.items():
   if i==ma: last=a
   
# we keep last  !!  
# do it again !!

 a=last
 dc={}
 dc[a]=0
 for b in d[a]: 
  if b in dc.keys(): pass
  else: dc[b]=1
  
#Initialize S ans S1, Start iterating
 S=d[a]
 comp=[]
 S1=set([a])
 S=S.union(S1)
#print("S=",S)
 while S > S1:
    S1=S
    for u in S:
       S=S.union(d[u])
       for v in d[u]:
         if v in dc.keys(): dc[v]=min(dc[v],dc[u]+1)
         else: dc[v]=dc[u]+1
 dist=list(dc.values())
 ma=max(dist)
 for a, i in dc.items():
   if i==ma: last=a

 return(max(dist))
TICKER_API_URL = 'https://api.coinmarketcap.com/v1/ticker/'
def get_latest_crypto_price(crypto):
  
  response = requests.get(TICKER_API_URL+crypto)
  response_json = response.json()
  
  return float(response_json[0]['price_usd'])
def step_reservoir_sampling(edge):
    global init, w_i, w_1, M, window_counter, lamb, t_i, dict_test, tau, t,hh
    if init == 0:
        # initialise w_i
        w_1 = int(edge[2])+tau
        w_i = int(edge[2])+tau
        t_i = int(edge[2])+lamb
        hh = []
        # window counter
        window_counter = 1
        # terminate initialisation
        init+=1
    # check if edge is indide w_i
    if int(edge[2]) < t_i:
        M[-1]+=1
    if int(edge[2]) >= t_i:
        print(M)
        #lst =[]
        if int(edge[2]) >= w_1:
          comp = components(reservoir)
          comp1 = sorted(comp, reverse=True)
          xx = comp_edges(comp1)
          r = requests.get('https://api.coindesk.com/v1/bpi/currentprice.json')
          bitcoin_data = dict(r.json())
          bitcoin_value = bitcoin_data["bpi"]["USD"]["rate_float"]
          #Import the requests library
          
          x = comp1[0][0]
          y = comp1[0][1]
          z = diameter(0)
          t= window_counter
          #ticker = "^FCHI"
          url = 'https://in.finance.yahoo.com/quote/' + ticker
          url1 = 'https://in.finance.yahoo.com/quote/' + ticker1
          session = requests_html.HTMLSession()
          session1 = requests_html.HTMLSession()
          r = session.get(url)
          r1 = session1.get(url1)
          content1 = BeautifulSoup(r1.content, 'lxml')
          content = BeautifulSoup(r.content, 'lxml')
          uu1 = content1.select_one('.Mb\(-4px\)').text
          uu1 = float(uu1.replace(',',''))
          uu = content.select_one('.Mb\(-4px\)').text
          uu = float(uu.replace(',',''))
          ll =[(t,x,y,z,uu)]
          ll1 =[(t,x,y,z,uu,uu1)]
          if window_counter >= 1:
            hh.append(ll1)
          else:
            print("")

          with open('sample_data/%s_step_reservoir_edges.csv', 'a') as f:
            #f.write("window_counter,vertices,edges,Diameter,bitcoin_value\n")
            writer = csv.writer(f, delimiter=',')
            writer.writerows(ll1)
            f.close()
          if window_counter == 5:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M3",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          elif window_counter ==  12:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])

            
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          elif window_counter == 22:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])

            
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          elif window_counter == 32:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])

            
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          elif window_counter == 42:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])

            
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          elif window_counter == 52:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])

            
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          elif window_counter == 62:
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
            df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])

            fig = go.Figure([go.Scatter(x=df['colA'], y=df['colC'], name="arêtes")])

            
            fig.add_trace(
              go.Scatter(x=df['colA'], y=df['colD'], name="Diameter" ),
              
            )
            
            fig.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig.show()
            fig1 = go.Figure([go.Scatter(x=df['colA'], y=df['colE'],name="ticker value" )])
            fig1.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig1.show()
            fig2 = go.Figure([go.Scatter(x=df['colA'], y=df['colF'],name="ticker2 value" )])
            fig2.update_xaxes(
              dtick="M1",
              tickformat="")
            # Set y-axes titles
            fig2.show()
          else:
            print("")

            
  
          #print("Actual Bitcoin pricing ", bitcoin_value, "$") 
          #f window_counter == 3:
           #x = comp1[0][0]
           #y = comp1[0][1]
           #z = diameter(0)
           #t= window_counter
           #print("Diameter =",z,",","vertices =",x,",","edges=", y,",","time  =",t, end='\n')
          #else:
           #x = comp1[0][0]
           #y = comp1[0][1]
           #z = diameter(0)
           #t= window_counter
           #print(z,x,y,t,sep=',', end='\n')
          #print("Diameter of largest component  =",diameter(0))
          
          #f.write("Diameter,vertices,edges,time\n")
          #f.close()
          
          index = 0
          for i in comp1:
              if int(i[0]) >= int(threshold):
                  #write_edge_reservoir(hh,time_export)
                  index+=1
              else:
                  break
        while len(reservoir) > 0 and int(reservoir[0][2]) < w_i-tau:
            # while the previous condition is satisfy do
            del reservoir[0]
        window_counter +=1
        del M[0]
        M.append(1)
        t_i += lamb
        w_i += lamb
        while len(reservoir) < 0 and int(reservoir[0][2]) > w_i-tau:
          print(hh)
    if sum(M) < k:
        reservoir.append(edge)
    else:
        j = random.randint(0, sum(M))
        if j < k:
            if len(reservoir) < k:
                reservoir.append(edge)
            else:
                del reservoir[j]
                reservoir.append(edge)
    
    return reservoir


# Get date
now = datetime.datetime.now()
def ask_exit():
    for task in asyncio.Task.all_tasks():
        task.cancel()
async def consume(queue):
    while True:
        # wait for an item from the producer
        item = await queue.get()
        twitter_edges_graph = create_graph(item)
        queue.task_done()
        global time_counter, global_counter, interval_counter
        if len(twitter_edges_graph) > 0:
            # process the item
            # print('consuming {}...'.format(twitter_edges_graph))
            # Notify the queue that the item has been processed
            for edge in twitter_edges_graph:
                if tracking[0] in edge[1].lower():
                    pass
                else:
                    global_counter+=1
                    interval_counter+=1
                    window_reservoir_sampling = step_reservoir_sampling(edge)
        if time.time()*1000 > start + timeout:
            print('Capture terminated')
            ask_exit()



class Client(PeonyClient):
    pass

# every class inheriting from `PeonyClient` or `BasePeonyClient` has
# an event_stream function that can be used on an `EventStream`
@Client.event_stream
class UserStream(EventStream):

    def stream_request(self):
        """
            The stream_request method returns the request
            that will be used by the stream
        """
        return self.stream.statuses.filter.post(track=tracking)


    # the on_connect event is triggered on connection to an user stream
    # https://dev.twitter.com/streaming/overview/messages-types#friends-lists-friends
    @events.on_connect.handler
    def connection(self, data):
        consumer = asyncio.ensure_future(consume(queue))
        print("Connected to stream!")

    # the on_tweet event is triggered when a tweet seems to be sent on
    # the stream, by default retweets are included
    @events.on_tweet.handler
    async def tweet(self, data):
        await queue.put(data)
        #print("producing")


if __name__ == '__main__':
    queue = asyncio.Queue()
    client = Client(consumer_key=consumer_key,
                     consumer_secret=consumer_secret,
                     access_token=access_token,
                     access_token_secret=access_token_secret)
    client.run()

Connected to stream!
[0, 0, 98]
[0, 98, 152]
[98, 152, 127]

[152, 127, 91]

[127, 91, 141]


[91, 141, 100]

[141, 100, 164]

[100, 164, 160]

[164, 160, 109]

[160, 109, 136]

[109, 136, 93]

[136, 93, 144]


[93, 144, 121]

[144, 121, 129]

[121, 129, 142]

[129, 142, 104]

[142, 104, 101]

[104, 101, 125]

[101, 125, 144]

[125, 144, 121]

[144, 121, 138]

[121, 138, 170]


[138, 170, 141]

[170, 141, 120]

[141, 120, 144]

[120, 144, 135]

[144, 135, 152]

[135, 152, 122]

[152, 122, 177]

[122, 177, 149]

[177, 149, 107]

[149, 107, 139]


[107, 139, 91]

[139, 91, 138]

[91, 138, 150]

[138, 150, 124]

[150, 124, 119]

[124, 119, 131]

[119, 131, 154]

[131, 154, 175]

[154, 175, 197]

[175, 197, 182]


[197, 182, 191]

[182, 191, 198]

[191, 198, 219]

[198, 219, 194]

[219, 194, 167]

[194, 167, 164]

[167, 164, 163]

[164, 163, 159]

[163, 159, 189]

[159, 189, 157]


[189, 157, 157]

[157, 157, 222]

[157, 222, 141]

[222, 141, 167]

[141, 167, 181]

[167, 181, 202]

[181, 202, 236]

[202, 236, 218]

[236, 218, 251]

[218, 251, 163]


[251, 163, 146]

[163, 146, 179]

[146, 179, 194]

[179, 194, 201]

[194, 201, 173]

Capture terminated


In [ ]:
# Using graph_objects
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])
fig = go.Figure([go.Scatter(x=df['colA'], y=df['colB'])])

# Add traces



fig.add_trace(
  go.Scatter(x=df['colA'], y=df['colD'], name="Diameter"),
)


fig.update_xaxes(
  dtick="M1",
  tickformat="")
fig.update_layout(
    template="plotly_dark"
)
            
fig.show()

In [ ]:
# Using graph_objects
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv')
df = pd.read_csv('sample_data/%s_step_reservoir_edges.csv', names=['colA', 'colB','colC','colD','colE','colF'])
fig = go.Figure([go.Scatter(x=df['colA'], y=df['colE'])])


fig.update_xaxes(
  dtick="M1",
  tickformat="")
fig.update_layout(
    template="plotly_dark"
)
            
fig.show()

In [9]:
!jupyter nbconvert --to html /content/TD_5456.ipynb


[NbConvertApp] WARNING | pattern u'/content/TD_5456.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--inplace
    